In [1]:
library(Seurat)
library(liana)
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggpubr)
library(cowplot)
library(pals)
library(rstatix)

set.seed(123)

root.dir <- "/project/bicistronic_carT_gbm_Jackie/"
figures.dir <- paste0(root.dir, "Final/", "Figures/")
processed.data.dir <- paste0(root.dir, "ProcessedData/")
csf.liana.output.dir <- paste0(processed.data.dir, "liana/", "csf/") # liana results (table) will be outputted to this folder
csf.liana.figures.dir <- paste0(figures.dir, "interactions/", "csf/") # figures will be outputted in this folder
dir.create(file.path(csf.liana.figures.dir, "supp")) # supplemental figures will be outputted in this subfolder

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘cowplot’


The following object is masked from ‘package:ggpubr’:

    get_legend



Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter


Warning message in dir.create(file.path(csf.liana.figures.dir, "supp")):
“'/project/bicistronic_carT_gbm_Jackie/Final/Figures/interactions/csf//supp' already exists”


In [2]:
# for plotting
col.pal <- stepped3(20)
all.patients <- c("P6", "P1", "P8", "P5", "P2", "P7", "P4", "P3")
col.pal.values <- c(col.pal[5], col.pal[6], col.pal[7], col.pal[8], col.pal[11], col.pal[3], col.pal[2], col.pal[1])
scale.color.patient <- scale_color_manual(breaks = all.patients, values = col.pal.values)
scale.fill.patient <- scale_fill_manual(breaks = all.patients, values = col.pal.values)

# Start here with CSF Seurat object

In [3]:
csf <- readRDS(paste0(root.dir, "ProcessedData/seurat/annotated_prepost_csf_refined.rds"))
csf$response.status <- sapply(csf$Patient, function(x) ifelse(x %in% c('P2', 'P3', 'P7','P4'), 'Responder', 'Non-responder'))
csf$response.status <- factor(csf$response.status, levels = c("Non-responder", "Responder"))
csf$Day <- factor(csf$Day, levels = c("D0", "D7", "D21"))

In [4]:
# use broad monocyte/macrophage classification for liana
csf$cell_type_broad <- sapply(csf$cell_type, function(x) 
    case_when(
        x %in% c('Pro-inflammatory macrophage', 'CX3CR1+ monocyte', 'Inhibitory macrophage', 'Monocyte') ~ 'Mono/Mac',
        TRUE ~ x
    )
)
csf$cell_type <- csf$cell_type_broad
cell.type.col <- "cell_type_broad"
Idents(csf) <- cell.type.col

In [5]:
sample_ids = unique(csf$sample_id)
print(sample_ids)

 [1] "P1D0"  "P1D7"  "P1D21" "P2D0"  "P2D7"  "P2D21" "P3D0"  "P3D7"  "P3D21"
[10] "P4D0"  "P4D7"  "P4D21" "P5D0"  "P5D7"  "P5D21" "P6D0"  "P6D7"  "P6D21"
[19] "P7D0"  "P7D7"  "P7D21" "P8D0"  "P8D7"  "P8D21"


In [6]:
# run liana on each sample
for (sample_id_use in sample_ids) {
    print(sample_id_use)
    
    filename <- paste0(csf.liana.output.dir, sample_id_use, "_liana_test.tsv")

    # read in file if it already exists
    if (file.exists(filename)) {
        next
    }

    if (downsample) {
        csf_subset <- subset(csf, subset = sample_id == sample_id_use, downsample = n.downsample)
    } else {
        csf_subset = subset(csf, subset = sample_id == sample_id_use)
    }
    print(dim(csf_subset))
    print(table(csf_subset$cell_type_broad))
    
    liana_test <- liana_wrap(csf_subset, idents_col = cell.type.col)
    liana_test <- liana_test %>%
        liana_aggregate()
    write.table(liana_test, filename, sep = "\t", quote=F, col.names=T, row.names=F)
}

[1] "P1D0"
[1] "P1D7"
[1] "P1D21"
[1] "P2D0"
[1] "P2D7"
[1] "P2D21"
[1] "P3D0"
[1] "P3D7"
[1] "P3D21"
[1] "P4D0"
[1] "P4D7"
[1] "P4D21"
[1] "P5D0"
[1] "P5D7"
[1] "P5D21"
[1] "P6D0"
[1] "P6D7"
[1] "P6D21"
[1] "P7D0"
[1] "P7D7"
[1] "P7D21"
[1] "P8D0"
[1] "P8D7"
[1] "P8D21"


In [7]:
# read in LIANA results and aggregate all samples
filename <- paste0(csf.liana.output.dir, "all_samples_combined_unfiltered_liana.tsv")
if (!file.exists(filename)) {
    
    liana_combined <- data.frame()
    for (sample_id_use in sample_ids) {
    
        liana_test <- read.csv(paste0(csf.liana.output.dir, sample_id_use, "_liana_test.tsv"), sep = "\t")
    
        patient_id <- unlist(str_split(sample_id_use, "D"))[1]
        day <- paste0("D", unlist(str_split(sample_id_use, "D"))[2])
    
        # add metadata
        liana_test$sample_id <- sample_id_use
        liana_test$Patient <- patient_id
        liana_test$Day <- day
    
        response.status <- 'progression'
        if (patient_id %in% c('P2', 'P3', 'P7','P4')) {
            response.status <- 'responder'
        }
        liana_test$response.status <- response.status
        liana_combined <- rbind(liana_combined, liana_test)
    }
    liana_combined <- liana_combined %>% mutate(label = paste0(source, ":", ligand.complex, " -> ", target, ":", receptor.complex))
    liana_combined <- liana_combined %>% mutate(interaction.label = paste0(ligand.complex, " -> ", receptor.complex))
    liana_combined$significance.level <- sapply(liana_combined$aggregate_rank, function(x) case_when(
        x <= 0.05 ~ "<= 0.05",
        x <= 0.1 ~ "<= 0.1",
        TRUE ~ "> 0.1"
    ))
    
    write.table(liana_combined, filename, sep = "\t", quote=F, col.names=T, row.names=F)
} else {

    # read in aggregated file if it already exists
    liana_combined <- read.csv(filename, sep = "\t")
}

In [8]:
liana_combined$Day <- factor(liana_combined$Day, levels=c("D0", "D7", "D21"))
liana_combined$response.status <- sapply(liana_combined$Patient, function(x) ifelse(x %in% c("P3", "P4", "P7", "P2"), "Responder", "Non-responder"))
liana_combined$response.status <- factor(liana_combined$response.status, levels = c("Non-responder", "Responder"))


In [9]:
# swap CTLA4 from receptor to ligand
swapped.df <- liana_combined %>% filter(receptor.complex == "CTLA4") %>% rename(target = source, source = target, receptor.complex = ligand.complex, ligand.complex = receptor.complex)
other.df <- liana_combined %>% filter(receptor.complex != "CTLA4")
liana_combined <- rbind(other.df, swapped.df)
liana_combined <- liana_combined %>% mutate(label = paste0(source, ":", ligand.complex, " -> ", target, ":", receptor.complex))
liana_combined <- liana_combined %>% mutate(interaction.label = paste0(ligand.complex, " -> ", receptor.complex))

In [10]:
immune.checkpoints <- list(c("TGFB1", "TGFBR1_TGFBR2"),
                           c("BTLA", "TNFRSF14"),
                           c("TNFSF18", "TNFRSF18"),
                           c("CTLA4", "CD80"),
                           c("CTLA4", "CD86"),
                           c("ENTPD1", "ADORA2A"),
                           c("CD274", "PDCD1"),
                           c("PDCD1LG2", "PDCD1"),
                           c("LGALS9", "HAVCR2"),
                           c("HLA-DPA1", "LAG3"),
                           c("CD80", "CD28"),
                           c("CD86", "CD28"),
                           c("CD40LG", "CD40"),
                           c("TNFSF4", "TNFRSF4"), # OX40
                           c("TNFSF9", "TNFRSF9"), # 41BB
                           c("ICOSLG", "CD278"), # ICOS
                           c("CD70", "CD27")
                          )
canonical.interactions <- sapply(immune.checkpoints, function(x) paste0(x[1], " -> ", x[2])) 

In [11]:
plot.canonical.interactions <- function(liana_df, filepath, cell.types, interactions.use=immune.checkpoints, 
                                        levels.interactions=canonical.interactions, 
                                        levels.celltypes = NA,
                                        facet.by.source=T, timepoint.var = "Day",
                                        plot.width = 30, plot.height = NA) {

    
    plot.list <- list()
    for (s in cell.types) {

        if (facet.by.source) {
            liana_filter_df <- liana_df %>% dplyr::filter(source==s)
        } else {
            liana_filter_df <- liana_df %>% dplyr::filter(target==s)
        }
        
        interactions.df <- data.frame()
        for (i in 1:length(interactions.use)) {
            ligand <- interactions.use[[i]][1]
            receptor <- interactions.use[[i]][2]
            temp <- liana_filter_df %>% dplyr::filter(ligand.complex == ligand, receptor.complex == receptor)
            interactions.df <- rbind(interactions.df, temp)
        }
        # add 0 for missing data
        interactions.df <- interactions.df %>% complete(source, target, nesting(ligand.complex, receptor.complex, interaction.label), 
                                                        .data[[timepoint.var]], nesting(Patient, response.status), fill = 
                                                        list(sca.LRscore = 0, natmi.edge_specificity = 0))


        interactions.df$y <- interactions.df$sca.LRscore 
    
        df <- interactions.df 

        if(nrow(interactions.df) == 0) {
            next
        }
        
        if (!all(is.na(levels.interactions))) {
            df$interaction.label <- factor(df$interaction.label, levels = canonical.interactions)
        }

        if (!all(is.na(levels.celltypes))) {
            df$target <- factor(df$target, levels = levels.celltypes)
        }
    
        df$`Interaction strength` <- df$sca.LRscore 
        df$`Interaction specificity` <- df$natmi.edge_specificity
        df$Patient <- factor(df$Patient, levels = all.patients)

        if (facet.by.source) {
            x <- "target"
            x.label <- "Target"
        } else {
            x <- "source"
            x.label <- "Source"
        }

        p <- ggplot(df, aes(.data[[x]], interaction.label)) +
            geom_point(aes(
                           fill = `Interaction strength`, 
                           size = `Interaction specificity`),
                      shape = 21, stroke = 0.8) +
            theme_bw() +
            theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
            scale_fill_viridis_c(limits = c(0, 1)) +
            scale_size(
                limits = c(0, 1),
                range = c(0, 10),    
                breaks = c(0, 0.2, 0.4, 0.6, 0.8, 1),
            ) +
            facet_grid(cols=vars(Patient), rows=vars(.data[[timepoint.var]])) +
            xlab(x.label) +
            ylab("Interaction")

        plot.list[[s]] <- p
    }

    if (length(plot.list) == 0) {
        print("no interactions")
    } else {
        if (length(plot.list) > 1) {                                                              
            p <- plot_grid(plotlist = plot.list, ncol = 1, labels = names(plot.list))
        } else {
            p <- p + ggtitle(names(plot.list))
        }
        
        if (is.na(plot.height)) {
            plot.height <- length(plot.list) * 10
        }
        pdf(filepath, width = plot.width, height = plot.height)
        print(p)
        dev.off()
    }
}

In [12]:
plot.canonical.interactions.patients.aggregated <- function(liana_df, filepath, cell.types, timepoint.of.interest, interactions.use=immune.checkpoints, 
                                        levels.interactions=canonical.interactions, 
                                        levels.celltypes = NA,
                                        facet.by.source=T, timepoint.var = "Day",
                                        plot.width = 30, plot.height = NA) {

    plot.list <- list()
    for (s in cell.types) {

        if (facet.by.source) {
            liana_filter_df <- liana_df %>% dplyr::filter(source==s)
        } else {
            liana_filter_df <- liana_df %>% dplyr::filter(target==s)
        }
        
        interactions.df <- data.frame()
        for (i in 1:length(interactions.use)) {
            ligand <- interactions.use[[i]][1]
            receptor <- interactions.use[[i]][2]
            temp <- liana_filter_df %>% dplyr::filter(ligand.complex == ligand, receptor.complex == receptor, .data[[timepoint.var]] == timepoint.of.interest)
            interactions.df <- rbind(interactions.df, temp)
        }
        # add 0 for missing data
        interactions.df <- interactions.df %>% complete(source, target, nesting(ligand.complex, receptor.complex, interaction.label), 
                                                        nesting(Patient, response.status), fill = 
                                                        list(sca.LRscore = 0, natmi.edge_specificity = 0))

        if(nrow(interactions.df) == 0) {
            next
        }

        df <- interactions.df %>% 
            group_by(source, target, ligand.complex, receptor.complex, interaction.label) %>% 
            summarize(mean.sca.LRscore = mean(sca.LRscore),
                      mean.natmi.edge_specificity = mean(natmi.edge_specificity)) %>%
            ungroup()
        
        df$y <- df$mean.sca.LRscore
                
        if (!all(is.na(levels.interactions))) {
            df$interaction.label <- factor(df$interaction.label, levels = canonical.interactions)
        }

        if (!all(is.na(levels.celltypes))) {
            df$target <- factor(df$target, levels = levels.celltypes)
        }
        
        df$`Interaction strength` <- df$mean.sca.LRscore 
        df$`Interaction specificity` <- df$mean.natmi.edge_specificity

        if (facet.by.source) {
            x <- "target"
            x.label <- "Target"
        } else {
            x <- "source"
            x.label <- "Source"
        }

        p <- ggplot(df, aes(.data[[x]], interaction.label)) +
            geom_point(aes(
                           fill = `Interaction strength`, 
                           size = `Interaction specificity`),
                      shape = 21, stroke = 0.8) +
            theme_bw() +
            theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
            scale_fill_viridis_c(limits = c(0, 1)) +
            scale_size(
                limits = c(0, 0.1),
                range = c(0, 8),    
                breaks = c(0, 0.025, 0.05, 0.075, 0.1), # the size scaling can be customized as needed
            ) +
            ylab("Interaction") +
            xlab(x.label)
    
        plot.list[[s]] <- p
    }
    
    if (length(plot.list) == 0) {
        print("no interactions")
    } else {
        if (length(plot.list) > 1) {                                                              
            p <- plot_grid(plotlist = plot.list, ncol = 1, labels = names(plot.list))
        } else {
            p <- p + ggtitle(paste0(names(plot.list), " interactions at ", timepoint.of.interest))
        }
        
        if (is.na(plot.height)) {
            plot.height <- length(plot.list) * 10
        }
        pdf(filepath, width = plot.width, height = plot.height)
        print(p)
        dev.off()
    }
}

In [13]:
# Supplemental Figure 7A
plot.canonical.interactions(liana_df=liana_combined, filepath = paste0(csf.liana.figures.dir, "supp/", "Supp 7A immune checkpoints Treg as source.pdf"), cell.types=c("Treg"))


pdf 
  2

In [14]:
# canonical Treg interactions
treg.interactions <- list(
                           c("CTLA4", "CD80"),
                           c("CTLA4", "CD86"),
                           c("LGALS9", "HAVCR2"),
                           c("CD70", "CD27")
                          )

In [15]:
celltypes.use <- c('B', 'NK', 'CD4+ T', 'CD8+ T', 'CD4+ CAR T', 'CD8+ CAR T', 'Cycling CD8+ T', 'Mono/Mac', 'cDC1', 'cDC2', 'mregDC', 'pDC')
liana_combined_filtered <- liana_combined %>% filter(target %in% celltypes.use)

In [16]:
# Figure 6G
plot.canonical.interactions.patients.aggregated(liana_df=liana_combined_filtered, filepath = paste0(csf.liana.figures.dir, "6G Day 7 Treg interactions Treg as source aggregated.pdf"), 
                            cell.types=c("Treg"), timepoint.of.interest="D7", interactions.use = treg.interactions, levels.celltypes = celltypes.use, plot.width=8, plot.height = 4)

`summarise()` has grouped output by 'source', 'target', 'ligand.complex',
'receptor.complex'. You can override using the `.groups` argument.


pdf 
  2

In [17]:
# Figure 6I
p <- DotPlot(csf, features = c("CTLA4"), group.by = "cell_type") + ggtitle("CTLA4 expression in CSF") + xlab("") + ylab("") + coord_flip() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))
pdf(paste0(csf.liana.figures.dir, "6I Treg CTLA4 gene expression.pdf"), width = 8, height = 5) 
print(p)
dev.off()

pdf 
  2

In [18]:
plot.specific.interaction.specific.timepoint <- function(df, filepath, ligand.of.interest, receptor.of.interest, 
                             source.of.interest, targets.of.interest, timepoint.of.interest, timepoint.var="Day",
                             plot.width = 6, plot.height = 6) {

    plot.list <- list()
    for (target.of.interest in targets.of.interest) {
        
        interactions.df <- df %>% filter(ligand.complex == ligand.of.interest, receptor.complex == receptor.of.interest, 
                                         source == source.of.interest, target == target.of.interest, .data[[timepoint.var]] == timepoint.of.interest)
        interactions.df$y <- interactions.df$sca.LRscore 
        plot.title <- paste0(timepoint.of.interest, ": ", source.of.interest, " ", ligand.of.interest, " -> ", receptor.of.interest, " ", target.of.interest)
        
        # add 0 for missing data
        interactions.df <- interactions.df %>% complete(source, target, nesting(ligand.complex, receptor.complex, interaction.label), nesting(Patient=all.patients, response.status=c(rep("Non-responder", 4), rep("Responder", 4))), fill = list(y = 0))
        
        # plot interaction strength
        stat.test <- interactions.df %>%
          wilcox_test(y ~ response.status) %>%
          adjust_pvalue(method = "fdr") %>%
          add_significance("p.adj") %>%
          add_y_position()
        
        p <- ggplot(interactions.df, aes(response.status, y)) +
            geom_boxplot(outlier.shape = NA) + 
            geom_point(aes(fill = Patient), 
                       position = position_jitterdodge(jitter.width=0.2),
                       shape = 21, stroke = 1
                      ) + 
            theme_bw() +
            xlab("Response status") +
            ylab("Interaction strength") +
            scale.fill.patient +
            ggtitle(plot.title) +
            stat_pvalue_manual(stat.test, label = "p.adj", tip.length = 0)
                
        plot.list[[target.of.interest]] <- p
    }

    if (length(plot.list) > 1) {
        p <- plot_grid(plotlist = plot.list)
    }
    
    pdf(filepath, width = plot.width, height = plot.height)
    print(p)
    dev.off()

}

In [19]:
# Figure 6H
plot.specific.interaction.specific.timepoint(df=liana_combined, filepath=paste0(csf.liana.figures.dir, "6H Day 7 Treg LGALS9 -> HAVCR2 MonoMac.pdf"), 
                 ligand.of.interest="LGALS9", receptor.of.interest="HAVCR2", source.of.interest="Treg", 
                 targets.of.interest = "Mono/Mac", timepoint.of.interest = "D7")

pdf 
  2

In [20]:
plot.gene.expression.specific.timepoint <- function(seurat.obj, gene.of.interest, celltypes.use, timepoint.of.interest, filepath, timepoint.var = "Day", plot.width=6, plot.height=6) {

    plot.list <- list()
    for (celltype in celltypes.use) {
    
        seurat.obj$gene.of.interest <- seurat.obj[[gene.of.interest]]
        gene.df <- seurat.obj@meta.data %>% 
            filter(cell_type == celltype, .data[[timepoint.var]] == timepoint.of.interest) %>%
            group_by(sample_id, Patient, response.status) %>% 
            summarize(mean.expression = mean(gene.of.interest)) %>%
            ungroup()

        stat.test <- gene.df %>%
          wilcox_test(mean.expression ~ response.status) %>%
          adjust_pvalue(method = "fdr") %>%
          add_significance("p.adj") %>%
          add_y_position()
        
        p <- ggplot(gene.df, aes(response.status, mean.expression)) +
            geom_boxplot(outlier.shape = NA) + 
            geom_point(aes(fill = Patient), 
                       position = position_jitterdodge(jitter.width=0.2),
                       shape = 21, stroke = 1
                      ) + 
            theme_bw() +
            scale.fill.patient +
            ylab(gene.of.interest) +
            xlab("Response status") +
            ggtitle(paste0(gene.of.interest, " expression in ", celltype, " at ", timepoint.of.interest)) +
            stat_pvalue_manual(stat.test, label = "p.adj", tip.length = 0)
        
        plot.list[[celltype]] <- p
    }

    p <- plot_grid(plotlist = plot.list)
    pdf(filepath, width=plot.width, height=plot.height)
    print(p)
    dev.off()
    
}

In [21]:
data <- FetchData(csf, vars = c("LGALS9", "HAVCR2"))
csf <- AddMetaData(csf, data)

# Supplemental Figure 7B
plot.gene.expression.specific.timepoint(seurat.obj=csf, gene.of.interest = "LGALS9", celltypes.use = "Treg", timepoint.of.interest = "D7", filepath = paste0(csf.liana.figures.dir, "supp/", "Supp 7B LGALS9 expression on Treg Day7.pdf")) 

# Supplemental Figure 7C
plot.gene.expression.specific.timepoint(seurat.obj=csf, gene.of.interest = "HAVCR2", celltypes.use = "Mono/Mac", timepoint.of.interest = "D7", filepath = paste0(csf.liana.figures.dir, "supp/", "Supp 7C HAVCR2 expression on MonoMac Day7.pdf"))


`summarise()` has grouped output by 'sample_id', 'Patient'. You can override
using the `.groups` argument.


pdf 
  2

`summarise()` has grouped output by 'sample_id', 'Patient'. You can override
using the `.groups` argument.


pdf 
  2

# Plot exhaustion/activation signatures

In [22]:
wherry_signatures <- read.csv(paste0(root.dir, "Resources/", "wherry_pace_signatures.csv"))
effector.signature <- wherry_signatures %>% pull(Pace_effector_markers)
effector.signature <- effector.signature[effector.signature != ""]
exhaustion.signature <- wherry_signatures %>% pull(Wherry_exhaustion_markers)

In [23]:
gene.signatures <- list()
gene.signatures[["effector.signature"]] <- effector.signature
gene.signatures[["exhaustion.signature"]] <- exhaustion.signature
csf <- AddModuleScore(csf, feature=gene.signatures, name=names(gene.signatures), search = T)
csf@meta.data <- csf@meta.data %>% rename(effector.signature=effector.signature1, exhaustion.signature=exhaustion.signature2)

csf$response.status <- sapply(csf$Patient, function(x) ifelse(x %in% c("P3", "P4", "P7", "P2"), "Responder", "Non-responder"))
csf$response.status <- factor(csf$response.status, levels = c("Non-responder", "Responder"))

Warning message:
“The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.
ℹ The deprecated feature was likely used in the Seurat package.
  Please report the issue at <https://github.com/satijalab/seurat/issues>.”
Warning message:
“The following features are not present in the object: CCL9, KLRA9, ATPIF1, attempting to find updated synonyms”
Found updated symbols for 1 symbols

ATPIF1 -> ATP5IF1

Warning message:
“The following features are still not present in the object: CCL9, KLRA9”
Warning message:
“The following features are not present in the object: GP49B, GLYCOP, TNFSF6, MOX2, GPR56, PRKWNK, ITPR5, G1P2, ICSBP1, SERPINA, TCRGG-V4, TCRB-V1, ZFP9H1, HIST1H2, CAR2, CYP4V3, C76628, SFRS7, MTV43, C79248, PENK1, A43010, SEPTIN, RCN, WBP5, PBEF1, KDT1, HRB, TUBB2, SCL29A1, attempting to find updated synonyms”
Found updated symbols for 11 symbols

TNFSF6 -> FASLG
MOX2 -> CD200
GPR56 -> ADGRG1
G1P2 -> ISG15
ICSBP1 -> IRF

In [24]:
data <- FetchData(csf, vars = c("HLA-DRA", "ENTPD1", "CTLA4", "PDCD1"))
csf <- AddMetaData(csf, data)

In [25]:
# Figure 6J
genes.of.interest <- c("HLA-DRA", "ENTPD1", "CTLA4",  "effector.signature", "PDCD1", "exhaustion.signature")

plot.list <- list()
celltype <- "CD8+ T"
day <- "D7"
timepoint.var <- "Day"
seurat.obj <- csf

for (gene.of.interest in genes.of.interest) {

    seurat.obj$gene.of.interest <- seurat.obj[[gene.of.interest]]
    gene.df <- seurat.obj@meta.data %>% 
        filter(cell_type == celltype, .data[[timepoint.var]] == day) %>%
        group_by(cell_type, sample_id, Patient, response.status) %>% 
        summarize(mean.expression = mean(gene.of.interest)) %>%
        ungroup()
    
    stat.test <- gene.df %>%
      wilcox_test(mean.expression ~ response.status) %>%
      adjust_pvalue(method = "fdr") %>%
      add_significance("p.adj") %>%
      add_y_position()


    if (gene.of.interest == "effector.signature") {
        gene.of.interest <- "Effector signature"
    } else if (gene.of.interest == "exhaustion.signature") {
        gene.of.interest <- "Exhaustion signature"
    }
    
    p <- ggplot(gene.df, aes(response.status, mean.expression)) +
        geom_boxplot(outlier.shape = NA) + 
        geom_point(aes(fill = Patient), 
                   position = position_jitter(width=0.2),
                   shape = 21, stroke = 1
                  ) + 
        theme_bw() +
        scale.fill.patient +
        ylab("Expression level") +
        ggtitle(gene.of.interest) +
        stat_pvalue_manual(stat.test, label = "p.adj", tip.length = 0) +
        xlab("Response status")
    
    plot.list[[gene.of.interest]] <- p
}

p <- plot_grid(plotlist = plot.list)
pdf(paste0(csf.liana.figures.dir, "6J CD8+ T Day 7 marker expression.pdf"), width=12, height=9)
print(p)
dev.off()

`summarise()` has grouped output by 'cell_type', 'sample_id', 'Patient'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'cell_type', 'sample_id', 'Patient'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'cell_type', 'sample_id', 'Patient'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'cell_type', 'sample_id', 'Patient'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'cell_type', 'sample_id', 'Patient'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'cell_type', 'sample_id', 'Patient'. You
can override using the `.groups` argument.


pdf 
  2

In [26]:
sessionInfo() 

R version 4.3.2 (2023-10-31)
Platform: aarch64-unknown-linux-gnu (64-bit)
Running under: Ubuntu 22.04.3 LTS

Matrix products: default
BLAS:   /usr/lib/aarch64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/aarch64-linux-gnu/openblas-pthread/libopenblasp-r0.3.20.so;  LAPACK version 3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] rstatix_0.7.2      pals_1.10          cowplot_1.1.3      ggpubr_0.6.0      
 [5] ggplot2_3.5.2      tidyr_1.3.1        dplyr_1.1.4        liana_0.1.14      
